# Test Code for Unzipping Local Files

In [12]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from zipfile import ZipFile

import pyspark.sql.functions as F
import pyspark
from pyspark import SparkContext, SparkConf

import os

## General

In [3]:
sc.stop()
spark.stop()

NameError: name 'sc' is not defined

In [4]:
configure = SparkConf().setAppName('name').setMaster('local')
sc = SparkContext(conf = configure)

In [5]:
# getOrCreate() modifies existing spark session 
spark = SparkSession.builder.appName('local_name').config('config option', 'config value').getOrCreate()

## Song Data

In [15]:
with ZipFile('data/song-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data/local_song_data')

In [16]:
df = spark.read.json('data/local_song_data/song_data/*/*/*/*.json')

In [17]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [18]:
df.describe().show()

+-------+------------------+------------------+---------------+------------------+-----------+------------------+---------+------------------+--------------------+-----------------+
|summary|         artist_id|   artist_latitude|artist_location|  artist_longitude|artist_name|          duration|num_songs|           song_id|               title|             year|
+-------+------------------+------------------+---------------+------------------+-----------+------------------+---------+------------------+--------------------+-----------------+
|  count|                71|                31|             71|                31|         71|                71|       71|                71|                  71|               71|
|   mean|              null| 36.55297161290323|           null|-73.25123258064517|       null|239.72967605633804|      1.0|              null|                null|785.9577464788732|
| stddev|              null|12.431023413063544|           null| 36.05807592882607|       n

#### Create songs_table

In [19]:
songs_table = df['song_id', 'title', 'artist_id', 'year', 'duration']

#### Partition table and export to parquet file

In [20]:
output_data = 'output/'

songs_table.write.partitionBy('year', 'artist_id').mode('overwrite').parquet(os.path.join(output_data, 'songs'))

#### Create artists_table

In [23]:
artists_table = df['artist_id', 'artist_name', 'artist_location', 'artist_lattitude', 'artist_longitude']

AnalysisException: "cannot resolve '`artist_lattitude`' given input columns: [artist_name, song_id, year, artist_id, artist_longitude, num_songs, artist_latitude, duration, artist_location, title];;\n'Project [artist_id#645, artist_name#649, artist_location#647, 'artist_lattitude, artist_longitude#648]\n+- Relation[artist_id#645,artist_latitude#646,artist_location#647,artist_longitude#648,artist_name#649,duration#650,num_songs#651L,song_id#652,title#653,year#654L] json\n"

## Log Data

In [2]:
with ZipFile('data/log-data.zip', 'r') as zip_ref:
    zip_ref.extractall('data/local_log_data')

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
log_df = spark.read.json('data/local_log_data')

In [5]:
log_df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
log_df.describe().show()

+-------+------------------+----------+---------+------+------------------+--------+------------------+-----+--------------------+------+-------+--------------------+------------------+--------------------+------------------+--------------------+--------------------+-----------------+
|summary|            artist|      auth|firstName|gender|     itemInSession|lastName|            length|level|            location|method|   page|        registration|         sessionId|                song|            status|                  ts|           userAgent|           userId|
+-------+------------------+----------+---------+------+------------------+--------+------------------+-----+--------------------+------+-------+--------------------+------------------+--------------------+------------------+--------------------+--------------------+-----------------+
|  count|              6820|      8056|     7770|  7770|              8056|    7770|              6820| 8056|                7770|  8056|   80

NameError: name 'songs' is not defined